<a href="https://colab.research.google.com/github/Saim-Hassan786/OpenAI-SDK-Input-Output-Guardrails/blob/main/Input_And_Output_Guardrails_OpenAI_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input And Output Guardrails With OpenAI Agents SDK

In [2]:
!pip install -Uq openai-agents

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [13]:
from agents import (
    Agent,
    Runner,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    TResponseInputItem,
    RunContextWrapper,
    GuardrailFunctionOutput,
    input_guardrail,
    InputGuardrailTripwireTriggered,
    output_guardrail,
    OutputGuardrailTripwireTriggered,
)

In [12]:
from openai import AsyncOpenAI
from pydantic import BaseModel

external_client = AsyncOpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=GOOGLE_API_KEY,
)
set_default_openai_client(external_client)
set_tracing_disabled(True)
set_default_openai_api("chat_completions")

# Input Guardrails Code

In [27]:
class MathHomework(BaseModel):
  reasoning : str
  is_math_homework : bool

input_guardrail_agent = Agent(
    name = "Input Guardrail Agent",
    instructions="Check if the User input is related to a math question or not",
    model = "gemini-2.0-flash",
    output_type=MathHomework
)

@input_guardrail
async def check_math_query(context:RunContextWrapper,agent:Agent,input:str | list[TResponseInputItem])-> GuardrailFunctionOutput:
  result = await Runner.run(
      input_guardrail_agent,
      input = input,
      context = context.context
  )
  final_result = result.final_output_as(MathHomework)
  return GuardrailFunctionOutput(
      output_info = final_result,
      tripwire_triggered = final_result.is_math_homework
  )

main_agent = Agent(
    name= "Customer Support Agent",
    instructions="You are responsible for answering the user queries",
    model = "gemini-2.0-flash",
    input_guardrails=[check_math_query]
)

async def main():
  try :
    result = await Runner.run(
        main_agent,
        "What is the exponent in mathematics"
    )
    print(result.final_output)
  except InputGuardrailTripwireTriggered as e:
    print("Mathematical Questions are not Allowed")

In [28]:
import asyncio
asyncio.run(main())

Mathematical Questions are not Allowed


# Output Guardrails Code

In [33]:
class Response(BaseModel):
  response : str

class Mathoutput(BaseModel):
  reasoning : str
  is_math_homework : bool

output_guardrail_agent = Agent(
    name = "Output Guardrail Agent",
    instructions="Check if the User input is related to a math question or not",
    model = "gemini-2.0-flash",
    output_type=Mathoutput
)

@output_guardrail
async def check_math_output(context:RunContextWrapper, agent:Agent, output: Response)-> GuardrailFunctionOutput:
  result = await Runner.run(
      output_guardrail_agent,
      input = output.response,
      context = context.context
  )
  return GuardrailFunctionOutput(
      output_info = result.final_output,
      tripwire_triggered = result.final_output.is_math_homework
  )

main_agent_2 = Agent(
    name= "Customer Support Agent",
    instructions="You are responsible for answering the user queries",
    model = "gemini-2.0-flash",
    output_guardrails=[check_math_output],
    output_type=Response
)

async def main_1():
  try:
    result = await Runner.run(
        main_agent_2,
        "What is the exponent in mathematics"
    )
    print(result.final_output)
  except OutputGuardrailTripwireTriggered as e:
    print("Mathematical Question Outputs are not Allowed")


In [34]:
asyncio.run(main_1())

Mathematical Question Outputs are not Allowed
